## 03. Model Training and Evaluation

In [ ]:
from stt import CommonVoiceDataset, DatasetPreprocessor, Evaluator, DataCollatorSpeechSeq2SeqWithPadding, Evaluator, training_args
from utils import MODEL_NAME, SIMILAR_TARGET_LANGUAGE, TASK, MAX_INPUT_LENGTH
from utils import NAME_DATASET, TARGET_LANGUAGE, PROCESSED_DATA_PATH
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

In [ ]:
preprocessor = DatasetPreprocessor(MODEL_NAME, SIMILAR_TARGET_LANGUAGE, TASK, MAX_INPUT_LENGTH)
normalizer = BasicTextNormalizer()

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=preprocessor.processor)
evaluator = Evaluator(processor=preprocessor.processor, normalizer=normalizer)

### Loading Pre-processed Dataset

In [ ]:
dataset = CommonVoiceDataset(name_dataset=NAME_DATASET, language=TARGET_LANGUAGE)

In [ ]:
dataset.load_from_disk(PROCESSED_DATA_PATH)

In [ ]:
dataset = dataset.get_dataset()

In [ ]:
dataset

### Loading Pre-Trained Checkpoint

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language=SIMILAR_TARGET_LANGUAGE, task=TASK, use_cache=True
)

### Define the Training Configuration


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=evaluator.compute_metrics,
    tokenizer=preprocessor.processor,
)

In [24]:
trainer.train()


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7efd55244ed0>>
Traceback (most recent call last):
  File "/root/miniconda3/envs/speech-to-text/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
